In [27]:
import requests
import time
import json
import locale

from datetime import date, datetime, timedelta

import pandas as pd
from bs4 import BeautifulSoup

# Домашнее задание к лекции "Основы веб-скрапинга и работы с API"

## Задание 1. 

### Обязательная часть

Будем парсить страницу со свежеми новостям на [habr.com/ru/all/](https://habr.com/ru/all/).

Вам необходимо собирать только те статьи, в которых встречается хотя бы одно требуемое ключевое слово. Эти слова определяем в начале кода в переменной, например:

`KEYWORDS = ['python', 'парсинг']`

 Поиск вести по всей доступной preview-информации (это информация, доступная непосредственно с текущей страницы). 
 
В итоге должен формироваться датафрейм вида: `<дата> - <заголовок> - <ссылка>`

In [85]:
PAGES = 10
KEYWORDS = set(['python', 'c++', 'парсинг', 'apple', 'linux', 'windows'])

In [86]:
posts = []
for i in range(0, PAGES):
    req = requests.get('https://habr.com/ru/all/' + ('page%d/' % i if i > 0 else ''))
    soup = BeautifulSoup(req.text, 'html.parser')
    posts = posts + [post for post in soup.find_all('article', class_='post')]

In [29]:
locale.setlocale(locale.LC_TIME, ('ru', 'UTF-8'))

'ru_RU.UTF-8'

In [87]:
def parse_date(date_str):
    result = None
    date_parts = date_str.split(' в ')
    if date_parts[0] == 'сегодня':
        result = datetime.now()
    elif date_parts[0] == 'вчера':
        result = datetime.now() - timedelta(days=1)
    else:
        result = datetime.strptime(date_parts[0], '%d %B %Y')
    hh, mm = date_parts[1].split(':')
    result = result.replace(hour=int(hh), minute=int(mm), second=0, microsecond=0)
    return result

In [88]:
def parse_post(post):
    post_date = parse_date(post.find('span', class_='post__time').text)
    post_link = post.find('a', class_='post__title_link').get('href')
    post_title = post.find('h2', class_='post__title').text
    post_preview = post.find('div', class_='post__text').text
    return {'date': post_date, 'title': post_title, 'link': post_link, 'text': post_preview}

In [89]:
def filter_posts(posts):
    result = pd.DataFrame()
    for post in posts:
        post = parse_post(post)
        if set(post['text'].lower().split(' ')).intersection(KEYWORDS):
            result = pd.concat([result, pd.DataFrame([post])])
    return result

In [90]:
result = filter_posts(posts)
result

,date,title,link,text
0,2021-02-10 12:36:00,\nПревращаем старый Amazon Kindle в платформу ...,https://habr.com/ru/company/vdsina/blog/541616/,\n\n\r\nМне всегда хотелось заполучить экран н...
0,2021-02-10 11:30:00,\nСражение Linux на поприще Windows\n,https://habr.com/ru/post/541730/,"\nВы замечали, что за 20 лет доля Linux систем..."
0,2021-02-10 11:00:00,"\nApple M1 хорош, но насколько он хорош в трас...",https://habr.com/ru/company/pixonic/blog/541622/,"\nЧип Apple M1, доступный в новых MacBook Air,..."
0,2021-02-10 12:36:00,\nПревращаем старый Amazon Kindle в платформу ...,https://habr.com/ru/company/vdsina/blog/541616/,\n\n\r\nМне всегда хотелось заполучить экран н...
0,2021-02-10 11:30:00,\nСражение Linux на поприще Windows\n,https://habr.com/ru/post/541730/,"\nВы замечали, что за 20 лет доля Linux систем..."
0,2021-02-10 11:00:00,"\nApple M1 хорош, но насколько он хорош в трас...",https://habr.com/ru/company/pixonic/blog/541622/,"\nЧип Apple M1, доступный в новых MacBook Air,..."
0,2021-02-09 19:21:00,\nСброс пробега для RICOH SP 150SUw\n,https://habr.com/ru/post/541674/,\n Я не верю в теорию заговора производит...
0,2021-02-09 14:53:00,\nRoot cause анализ инцидентов на корреляциях ...,https://habr.com/ru/post/541610/,\nОдной из задач систем ИТ-мониторинга являетс...
0,2021-02-09 12:12:00,\n20 лет VideоLAN Client Player: 8 неочевидных...,https://habr.com/ru/company/ruvds/blog/541470/,"\n\r\n20 лет — большой срок, даже Windows не с..."
0,2021-02-09 10:59:00,\n11 друзей Sanic’а – собираем асинхронное веб...,https://habr.com/ru/company/domclick/blog/531254/,\n Рано или поздно маленькие приложения р...


### Дополнительная часть (необязательная)

Улучшить скрипт так, чтобы он анализировал не только preview-информацию статьи, но и весь текст статьи целиком.

Для этого потребуется получать страницы статей и искать по тексту внутри этой страницы.  

Итоговый датафрейм формировать со столбцами: `<дата> - <заголовок> - <ссылка> - <текст_статьи>`


In [91]:
def load_post(post_link):
    req = requests.get(post_link)
    soup = BeautifulSoup(req.text, 'html.parser')
    text = soup.find('div', class_='post__text').text
    return text

In [92]:
def parse_post_(post):
    post_date = parse_date(post.find('span', class_='post__time').text)
    post_link = post.find('a', class_='post__title_link').get('href')
    post_title = post.find('h2', class_='post__title').text
    post_preview = load_post(post_link)
    return {'date': post_date, 'title': post_title, 'link': post_link, 'text': post_preview}

In [93]:
def filter_posts_(posts):
    result = pd.DataFrame()
    for post in posts:
        post = parse_post_(post)
        if set(post['text'].lower().split(' ')).intersection(KEYWORDS):
            result = pd.concat([result, pd.DataFrame([post])])
        time.sleep(0.3)
    return result

In [101]:
# result2 = filter_posts_(posts)
result2

,date,title,link,text
0,2021-02-10 12:42:00,\nПопулярные заблуждения о C#\n,https://habr.com/ru/post/541786/,Эта статья является развёрнутым комментарием к...
0,2021-02-10 12:36:00,\nПревращаем старый Amazon Kindle в платформу ...,https://habr.com/ru/company/vdsina/blog/541616/,\n\r\nМне всегда хотелось заполучить экран на ...
0,2021-02-10 12:12:00,\nРаспознавание символов\n,https://habr.com/ru/post/541742/,Работа с изображениями — одна из самых распрос...
0,2021-02-10 11:30:00,\nСражение Linux на поприще Windows\n,https://habr.com/ru/post/541730/,"ВступлениеВы замечали, что за 20 лет доля Linu..."
0,2021-02-10 11:00:00,"\nApple M1 хорош, но насколько он хорош в трас...",https://habr.com/ru/company/pixonic/blog/541622/,"Чип Apple M1, установленный в новых MacBook Ai..."
0,2021-02-10 09:51:00,\nOSCP: как я сдавал самый известный экзамен п...,https://habr.com/ru/company/avito/blog/540738/,Offensive Security Certified Professional отли...
0,2021-02-10 09:24:00,\nУправление LCD и OLED дисплеями на AVR-ассем...,https://habr.com/ru/post/541648/,"Сразу предупреждаю, что не собираюсь разводить..."
0,2021-02-10 12:42:00,\nПопулярные заблуждения о C#\n,https://habr.com/ru/post/541786/,Эта статья является развёрнутым комментарием к...
0,2021-02-10 12:36:00,\nПревращаем старый Amazon Kindle в платформу ...,https://habr.com/ru/company/vdsina/blog/541616/,\n\r\nМне всегда хотелось заполучить экран на ...
0,2021-02-10 12:12:00,\nРаспознавание символов\n,https://habr.com/ru/post/541742/,Работа с изображениями — одна из самых распрос...


## Задание 2.

### Обязательная часть

Написать скрипт, который будет проверять список e-mail адресов на утечку при помощи сервиса [Avast Hack Ckeck](https://www.avast.com/hackcheck/).
Список email-ов задаем переменной в начале кода:  
`EMAIL = [xxx@x.ru, yyy@y.com]`

В итоге должен формироваться датафрейм со столбцами: `<почта> - <дата утечки> - <источник утечки> - <описание утечки>`  

**Подсказка**: сервис работает при помощи "скрытого" API. Внимательно изучите post-запросы.

In [98]:
EMAIL = ['xxx@ya.ru', 'yyy@gmail.com']

In [116]:
api_url = 'https://identityprotection.avast.com/v1/web/query/site-breaches/unauthorized-data'

headers = {
    'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.141 Safari/537.36',
    'Content-Type': 'application/json;charset=UTF-8',
    'Vaar-Header-App-Product': 'hackcheck-web-avast',
    'Vaar-Version': '0'
}

In [117]:
response = requests.post(api_url, json={'emailAddresses': EMAIL}, headers=headers)

In [ ]:
data = response.json()

In [133]:
result = pd.DataFrame()

for email, breach_ids in data['summary'].items():
    for breach_id in breach_ids['breaches']:
        breach_id = str(breach_id)
        row = {
            'email': email,
            'date': data['breaches'][breach_id]['publishDate'],
            'site': data['breaches'][breach_id]['site'],
            'desc': data['breaches'][breach_id]['description']
        }
        result = pd.concat([result, pd.DataFrame([row])])

result

,email,date,site,desc
0,xxx@ya.ru,2016-11-07T00:00:00Z,ir.netease.com,"In October 2015, the Chinese internet and gami..."
0,xxx@ya.ru,2020-07-30T00:00:00Z,hurb.com,"In March 2019, the Brazilian travel agency Her..."
0,xxx@ya.ru,2019-03-28T00:00:00Z,verifications.io,Big data e-mail verification platform verifica...
0,xxx@ya.ru,2020-05-21T00:00:00Z,vk.com,"At some time in 2020, the Russian social netwo..."
0,xxx@ya.ru,2016-11-01T00:00:00Z,qip.ru,"In 2011, Russian instant messaging service pro..."
...,...,...,...,...
0,yyy@gmail.com,2017-01-16T00:00:00Z,catchitenglish.com,"In September 2016, CatchItEnglish.com was brea..."
0,yyy@gmail.com,2017-10-29T00:00:00Z,jobstreet.com,Online recruitment site JobStreet was allegedl...
0,yyy@gmail.com,2017-05-26T00:00:00Z,leet.cc,"In February 2016, LEET's user database was all..."
0,yyy@gmail.com,2016-12-05T00:00:00Z,classroomcash.org,At an unknown date ClassroomCash.org was hacke...


### Дополнительная часть (необязательная)

Написать скрипт, который будет получать 50 последних постов указанной группы во Вконтакте.  
Документация к API VK: https://vk.com/dev/methods
, вам поможет метод [wall.get](https://vk.com/dev/wall.get)  
```
GROUP = 'netology'  
TOKEN = УДАЛЯЙТЕ В ВЕРСИИ ДЛЯ ПРОВЕРКИ, НА GITHUB НЕ ВЫКЛАДЫВАТЬ  
```

В итоге должен формироваться датафрейм со столбцами: `<дата поста> - <текст поста>`

In [142]:
api_url = 'https://api.vk.com/method/wall.get'
params = {
    'domain': 'netology',
    'access_token': '',
    'v': '5.126',
    'count': 50
}

In [143]:
response = requests.get(api_url, params=params)

In [146]:
data = pd.read_json(json.dumps(response.json()['response']['items']))

In [148]:
data[['date', 'text']]

,date,text
0,2021-02-05 07:53:00,Где работать дизайнеру? 🤔 \nГрафический дизайн...
1,2021-02-10 07:53:00,📆 15 февраля стартует бесплатный курс «Разрабо...
2,2021-02-09 14:57:00,"Раз, два, три — эмоция замри 🤪 \n \nЭмоциональ..."
3,2021-02-09 07:27:00,"Кто стоит за всеми приложениями, играми и сайт..."
4,2021-02-08 15:09:00,Сомнений больше нет: жизнь на удалёнке — есть ...
5,2021-02-08 07:38:00,"Если интернет сравнить с экзотической страной,..."
6,2021-02-07 13:07:00,*партнёрский пост*\n\n25-26 марта состоится ко...
7,2021-02-07 11:37:00,Можно положить кошелёк на подоконник в полнолу...
8,2021-02-06 12:07:00,*партнёрский пост* \n \nИздательство МИФ запус...
9,2021-02-06 08:07:00,"Собрали подборку статей, в помощь начинающим S..."


#### ПРИМЕЧАНИЕ
Домашнее задание сдается ссылкой на репозиторий [GitHub](https://github.com/).
Не сможем проверить или помочь, если вы пришлете:
- файлы;
- архивы;
- скриншоты кода.

Все обсуждения и консультации по выполнению домашнего задания ведутся только на соответствующем канале в slack.

##### Как правильно задавать вопросы аспирантам, преподавателям и коллегам?
Прежде чем задать вопрос необходимо попробовать найти ответ самому в интернете. Навык самостоятельного поиска информации – один из важнейших, и каждый практикующий специалист любого уровня это делает каждый день.

Любой вопрос должен быть сформулирован по алгоритму:  
1) Что я делаю?  
2) Какого результата я ожидаю?  
3) Как фактический результат отличается от ожидаемого?  
4) Что я уже попробовал сделать, чтобы исправить проблему?  

По возможности, прикрепляйте к вопросу скриншоты, либо ссылки на код. Оставляйте только проблемный и воспроизводимый участок кода, все решение выкладывать не допускается.
